# Recommender System Code Along Project

### 1. Import koniecznych pakietów i wstępna obróbka danych

Zaczynamy od importowania pakietu findspark, aby w dalszej kolejności umożliwić import Sparka i jego klas.

In [1]:
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('rec').getOrCreate()

Import danych z pliku CSV:

In [5]:
data = spark.read.csv('movielens_ratings.csv',  inferSchema=True, header=True)

In [6]:
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [8]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



Podział danych na zbiory treningowy i testowy:

In [9]:
train_set, test_set = data.randomSplit([0.8, 0.2])

### 2. Tworzenie i trening modelu

In [10]:
from pyspark.ml.recommendation import ALS

In [11]:
als = ALS(maxIter=5, regParam=0.01, userCol='userId', itemCol='movieId', ratingCol='rating')

In [12]:
model = als.fit(train_set)

In [13]:
predictions = model.transform(test_set)

In [15]:
predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|     31|   3.0|     8|  1.2487566|
|     31|   1.0|     0| -1.3071098|
|     85|   1.0|    28| -0.8291118|
|     85|   1.0|    23|-0.38921547|
|     85|   4.0|     7|   0.106143|
|     85|   1.0|    29|   3.278234|
|     85|   3.0|    21|    5.83236|
|     65|   1.0|     2|-0.20441227|
|     53|   1.0|    12|  -0.497562|
|     53|   3.0|    20|  1.9952193|
|     53|   1.0|    23|-0.99589896|
|     53|   5.0|    21|   4.107796|
|     53|   3.0|    14|  1.0012946|
|     78|   1.0|    20|0.017570507|
|     78|   1.0|    19|  0.2714653|
|     78|   1.0|     2| 0.12632082|
|     34|   3.0|     3|  3.2738307|
|     34|   1.0|    14|  0.4321177|
|     81|   1.0|    19|  1.7460945|
|     28|   1.0|    27| 0.55698556|
+-------+------+------+-----------+
only showing top 20 rows



### 3. Ocena modelu

In [16]:
from pyspark.ml.evaluation import RegressionEvaluator

In [17]:
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')

In [18]:
rmse = evaluator.evaluate(predictions)

In [19]:
print('RMSE')
print(rmse)

RMSE
2.2296557016051217


In [25]:
single_user = test_set.filter(test_set['userId']==11).select(['movieId', 'userId'])

In [26]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|     13|    11|
|     27|    11|
|     47|    11|
|     70|    11|
|     79|    11|
+-------+------+



In [27]:
recommendations = model.transform(single_user)

In [28]:
recommendations.orderBy('prediction', ascending=False).show()

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|     13|    11| 2.3545654|
|     47|    11| 1.9709153|
|     79|    11| 1.5069315|
|     27|    11| 1.2882609|
|     70|    11|-0.7039815|
+-------+------+----------+

